Notebook created to take in WikiArt data and parse movements

In [40]:
import os
import pandas as pd
import re

from cleaners.wikiart import *

In [2]:
output = pd.DataFrame(columns=['url','count'])
for i in os.listdir('./WikiArt Data/'):
    if i[-3:] == 'csv':
        chunk = pd.read_csv(f'./WikiArt Data/{i}', names=['url','count'],header=0)
        output = output.append(chunk, ignore_index=True)
        
output.drop('count',axis=1,inplace=True)
output.drop_duplicates(inplace=True)

In [3]:
def parse_artist(input: str) -> str:
    artist = re.findall('\/(.+)_',input)[0]
    # print(artist)
    artist_cap = [i.capitalize() for i in artist.split('-')]
    # print(artist_cap)
    output = artist_cap[0]
    output = ' '.join(artist_cap)
    return output

In [23]:
def process_title(input: str) -> str:
    w_year = '_([\w-]+)-\d{4}'
    no_year = '_([\w-]+).'
    
    if re.search('[0-9]', input) != None:
        title = re.findall(w_year, input)
    else:
        title = re.findall(no_year, input)
    
    if len(title) != 0:
        title_cap = [i.capitalize()  if i!='s' else "s" for i in title[0].split('-')]
        output = title_cap[0]
        output = ' '.join(title_cap).replace(" s", "'s") # probably a better way to do this...
        return output
    else:
        return 'review'

In [24]:
output['style'] = output.url.apply(lambda x: x.split('/')[0].replace('_',' '))
output['artist'] = output.url.apply(lambda x: parse_artist(x))
output['title'] = '-'
output['title'] = output.url.apply(lambda x: process_title(x))
re.purge()
re.purge()

In [25]:
[print(i) for i in output[output['title']=='review']['url'][0:10]]

Art_Nouveau_Modern/raphael-kirchner_deadly-sins-2.jpg
Impressionism/pierre-auguste-renoir_still-life-with-strawberries-2.jpg
Art_Nouveau_Modern/nicholas-roerich_polovtsian-camp-1.jpg
Romanticism/gustave-dore_the-inferno-canto-5-1.jpg
Romanticism/gustave-dore_don-quixote-81.jpg
Impressionism/childe-hassam_newfields-new-hampshire-1.jpg
Impressionism/claude-monet_low-tide-at-varengeville-02.jpg
Impressionism/claude-monet_anglers-on-the-seine-at-poissy(1).jpg
Romanticism/gustave-dore_paradise-lost-6.jpg
Naive_Art_Primitivism/marc-chagall_striking-the-rock-5.jpg


[None, None, None, None, None, None, None, None, None, None]

In [27]:
len(output), len(output[output['title']=='review']['url'])

(81446, 5582)

In [28]:
movements_ref = output[['artist','style']].drop_duplicates()

In [29]:
moma = pd.read_csv('./MoMA data/Artworks.csv')

In [30]:
overlap = [i.lower() for i in moma.Artist.unique() if i in movements_ref.artist.unique()]

In [31]:
len(overlap)

334

In [32]:
ov_count = [len(moma[moma.Artist==i]) for i in overlap]

In [33]:
len(ov_count), sum(ov_count)/len(moma)

(334, 0.0)

In [34]:
movements_ref

,artist,style
0,Rembrandt,Baroque
1,Paul Cezanne,Post Impressionism
2,Pierre Auguste Renoir,Impressionism
3,Ivan Aivazovsky,Romanticism
4,Boris Kustodiev,Art Nouveau Modern
...,...,...
164815,Octav Bancila,Naive Art Primitivism
164924,Marcel Janco,Naive Art Primitivism
164932,Frida Kahlo,Naive Art Primitivism
165052,James Ensor,Naive Art Primitivism


In [39]:
grouped_movements = movements_ref.groupby('artist')['style'].apply(', '.join).reset_index()
# taken from https://saturncloud.io/blog/how-to-use-groupby-to-concatenate-strings-in-python-pandas/#:~:text=Groupby%20is%20a%20powerful%20function,in%20one%20or%20more%20columns.

grouped_movements.to_csv('./WikiArt Data/movements_by_artist.csv', index=False)

In [36]:
movements_ref.artist.value_counts()[0:20]

Pablo Picasso           11
Salvador Dali            9
Henri Matisse            9
Kazimir Malevich         8
Martiros Saryan          7
Pyotr Konchalovsky       7
Georges Braque           7
William H. Johnson       7
Walter Battiss           7
James Ensor              6
Corneliu Michailescu     6
Umberto Boccioni         6
Jacek Malczewski         6
Ferdinand Hodler         6
Natalia Goncharova       6
Arthur Segal             6
Jean Metzinger           6
Hans Hofmann             6
Nicolae Vermont          6
Andre Derain             6
Name: artist, dtype: int64